# Data import

In [ ]:
import sqlite3
conn = sqlite3.connect('C:/JupData/mxm_dataset.db')
print(conn)

In [13]:
res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
print(res)

Metadata import

In [ ]:
conn_tmdb = sqlite3.connect('C:/JupData/track_metadata.db')

In [14]:
res.fetchall()

[('table', 'words', 'words', 2, 'CREATE TABLE words (word TEXT PRIMARY KEY)'),
 ('table',
  'lyrics',
  'lyrics',
  4,
  'CREATE TABLE lyrics (track_id, mxm_tid INT, word TEXT, count INT, is_test INT, FOREIGN KEY(word) REFERENCES words(word))')]

# Playing around in sql

In [15]:
res = conn.execute("SELECT word FROM words")
len(res.fetchall())

5000

In [74]:
res = conn.execute("SELECT word FROM words WHERE ROWID BETWEEN 50 AND 60")
res.fetchall()

[('out',),
 ('down',),
 ('get',),
 ('she',),
 ('was',),
 ('see',),
 ('if',),
 ('got',),
 ('never',),
 ('from',),
 ('he',)]

In [98]:
res = conn.execute("SELECT track_id FROM lyrics WHERE word='countdown' ORDER BY RANDOM() LIMIT 1")

In [99]:
res.fetchall()

[]

In [93]:
res = conn_tmdb.execute("SELECT artist_name, title FROM songs WHERE track_id='TRCKLDK12903CEA5A9'")
res.fetchall()

[('Old Crow Medicine Show', 'My Good Gal')]

# Got sick of SQL and switched to pandas

In [110]:
import pandas as pd
df = pd.read_sql_query("select * from lyrics limit 1000000;", conn)

In [111]:
print(df.shape)
print(df.tail())

(1000000, 5)
                  track_id  mxm_tid   word  count  is_test
999995  TRBMZBD128C7196BE5  1566461   kind      1        0
999996  TRBMZBD128C7196BE5  1566461   road      1        0
999997  TRBMZBD128C7196BE5  1566461   took      1        0
999998  TRBMZBD128C7196BE5  1566461   citi      1        0
999999  TRBMZBD128C7196BE5  1566461  reach      1        0


In [116]:
df_meta=pd.read_sql_query("SELECT * from songs limit 2000000;", conn_tmdb)

In [129]:
print(df_meta.shape)
print(df_meta.tail(1))

(1000000, 14)
                  track_id                                title  \
999999  TRYYYVU12903CD01E3  Fernweh feat. Sektion Kuchikäschtli   

                   song_id     release           artist_id  \
999999  SOWXJXQ12AB0189F43  So Oder So  AR7PLM21187B990D08   

                                 artist_mbid artist_name   duration  \
999999  3af2b07e-c91c-4160-9bda-f0b9e3144ed3       Texta  295.07873   

        artist_familiarity  artist_hotttnesss  year  track_7digitalid  \
999999            0.552977           0.454869  2004           8486723   

        shs_perf  shs_work  
999999        -1         0  


In [126]:
df_genres=pd.read_csv('C:\JupData\msd-topMAGD-genreAssignment.cls', delimiter="\t", header=None)

In [130]:
df_genres.columns=['track_id','genre']
print(df_genres.head(1))

             track_id     genre
0  TRAAAAK128F9318786  Pop_Rock


In [144]:
print(df_genres[df_genres['track_id']=='TRBMNGN12903CE41FA'])

                 track_id     genre
23465  TRBMNGN12903CE41FA  Pop_Rock


In [133]:
df_titles=df_meta.merge(df, on='track_id', how='right')

In [143]:
print(df_titles.shape, '\n', df_titles.iloc[10000])

(1000000, 14) 
 track_id                                TRBMNGN12903CE41FA
title                                  Never Met The Gooch
release                                       Kid Dynamite
artist_id                               ARQ6OS81187B998B73
artist_mbid           c5300096-cfb0-4801-b146-166e2d02f08a
artist_name                                   Kid Dynamite
duration                                           119.954
artist_familiarity                                0.640848
artist_hotttnesss                                 0.406503
year                                                  1998
track_7digitalid                                   7593181
word                                                   goe
count                                                    1
is_test                                                  0
Name: 10000, dtype: object


In [137]:
df_titles.drop(labels=['song_id', 'shs_perf', 'shs_work', 'mxm_tid'],axis=1, inplace=True)

In [138]:
df_merged=df_titles.merge(df_genres, on='track_id', how='left')

In [139]:
print(df_merged.shape)

(1000000, 15)


In [140]:
print(df_merged.head(50))

              track_id            title    release           artist_id  \
0   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
1   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
2   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
3   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
4   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
5   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
6   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
7   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
8   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
9   TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
10  TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
11  TRBMMVC128F429570F  Dead As History  Ire Works  ARMAC4T1187FB3FA4C   
12  TRBMMVC128F429570F  Dead As Histor

In [ ]:
conn.close()
conn_tmdb.close()